## Librerias y Funciones

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
#import plotly.express as px
#import plotly.graph_objects as go
from scipy import stats
from scipy.stats import spearmanr
from sklearn.preprocessing import LabelEncoder

In [2]:
def comparison_dist(data_1:pd.Series, data_2:pd.Series, action:str, x_label:str, fig_size:tuple):
    plt.style.use("bmh")    
    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=fig_size)

    #GRAFICO 1
    sns.kdeplot(x=data_1, ax=ax[0])
    ax[0].set_title('Distribucion original')
    ax[0].set_xlabel(x_label)

    #GRAFICO 2
    sns.kdeplot(data=data_2, ax=ax[1])
    ax[1].set_xlabel(x_label)
    ax[1].set_title(f'Distribucion despues de {action}') #ACTION(IMPUTAR, ELIMINAR)

    #PLOTEO
    plt.tight_layout()
    plt.show()
    plt.style.use("default")

## Conociendo los datos

### Exploracion del dataset

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/arielRas/DataSets/main/Hotel_Reservations.csv')
df.head(3)

,Booking_ID,num_adults,num_children,weekend_nights,week_nights,meal_plan,car_parking,room_type,lead_time,arrival_year,arrival_month,arrival_date,mkt_segment,repeated_guest,num_prev_cancellations,num_prev_not_canceled,avg_price_room,num_sp_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled


In [4]:
print(f'La cantidad de filas y columnas es: {df.shape}\n\n')
df.info()

La cantidad de filas y columnas es: (36275, 19)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Booking_ID              36275 non-null  object 
 1   num_adults              36275 non-null  int64  
 2   num_children            36275 non-null  int64  
 3   weekend_nights          36275 non-null  int64  
 4   week_nights             36275 non-null  int64  
 5   meal_plan               36275 non-null  object 
 6   car_parking             36275 non-null  int64  
 7   room_type               36275 non-null  object 
 8   lead_time               36275 non-null  int64  
 9   arrival_year            36275 non-null  int64  
 10  arrival_month           36275 non-null  int64  
 11  arrival_date            36275 non-null  int64  
 12  mkt_segment             36275 non-null  object 
 13  repeated_guest          36275 non-null  i

In [5]:
df.describe()

,num_adults,num_children,weekend_nights,week_nights,car_parking,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,num_prev_cancellations,num_prev_not_canceled,avg_price_room,num_sp_requests
count,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000
mean,1.844962,0.105279,0.810724,2.204300,0.030986,85.232557,2017.820427,7.423653,15.596995,0.025637,0.023349,0.153411,103.423539,0.619655
std,0.518715,0.402648,0.870644,1.410905,0.173281,85.930817,0.383836,3.069894,8.740447,0.158053,0.368331,1.754171,35.089424,0.786236
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2017.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,0.000000,17.000000,2018.000000,5.000000,8.000000,0.000000,0.000000,0.000000,80.300000,0.000000
50%,2.000000,0.000000,1.000000,2.000000,0.000000,57.000000,2018.000000,8.000000,16.000000,0.000000,0.000000,0.000000,99.450000,0.000000
75%,2.000000,0.000000,2.000000,3.000000,0.000000,126.000000,2018.000000,10.000000,23.000000,0.000000,0.000000,0.000000,120.000000,1.000000
max,4.000000,10.000000,7.000000,17.000000,1.000000,443.000000,2018.000000,12.000000,31.000000,1.000000,13.000000,58.000000,540.000000,5.000000


## Data Wrangling

### Variable ***num_adults***

Esta variable describe la cantidad de personas adultas que componen la reserva y que se alojaran en el hotel. Se puede divisar con el metodo ***describe()*** en la celda posterior que el valor minimo de la variable es cero y segun las reglas de negocio de este hotel, en las reservas debe haber al menos un adulto (>= 18 años) por reserva, los menores no pueden reservar por si solos. En el caso de que la reserva solo tenga menores, se interpreta el dato como erroneo.

In [6]:
df.num_adults.describe()

count    36275.000000
mean         1.844962
std          0.518715
min          0.000000
25%          2.000000
50%          2.000000
75%          2.000000
max          4.000000
Name: num_adults, dtype: float64

In [7]:
#CANTIDAD DE RESERVAS SIN ADULTOS
print(f'La cantidad de reservas sin adultos es: {df.loc[df.num_adults < 1].shape[0]}')

La cantidad de reservas sin adultos es: 139


In [8]:
#Se eliminan las reservas y se verifica
df = df.loc[df.num_adults >= 1]
print(f'La cantidad de reservas sin adultos despues de la modificacion es: {df.loc[df.num_adults < 1].shape[0]}')

#Se imprime informacion del dataframe despues de eliminar registros
print(f'La cantidad de filas y columnas del dataframe despues de la modificacion es: {df.shape}')

La cantidad de reservas sin adultos despues de la modificacion es: 0
La cantidad de filas y columnas del dataframe despues de la modificacion es: (36136, 19)


### Variable ***num_children***

Esta variable describe la cantidad de niños menores que componen la reserva y que se alojaran en el hotel. Se puede divisar con el metodo ***describe()*** en la celda posterior, que la variable muestra el valor **cero** para el tercer cuartil, lo que da la pauta de que en el 75% de los datos en esta variable es cero y su valor maximo es 10, como el valor maximo se aleja bastante del 3er cuartil, veremos si este valor es un outlier.

In [9]:
df.num_children.describe()

count    36136.000000
mean         0.097880
std          0.385097
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         10.000000
Name: num_children, dtype: float64

In [10]:
for item in range(0,11):
    print(f'La cantidad de reservas con {item} niños es: {df.loc[df.num_children == item].shape[0]}')

La cantidad de reservas con 0 niños es: 33577
La cantidad de reservas con 1 niños es: 1617
La cantidad de reservas con 2 niños es: 925
La cantidad de reservas con 3 niños es: 14
La cantidad de reservas con 4 niños es: 0
La cantidad de reservas con 5 niños es: 0
La cantidad de reservas con 6 niños es: 0
La cantidad de reservas con 7 niños es: 0
La cantidad de reservas con 8 niños es: 0
La cantidad de reservas con 9 niños es: 2
La cantidad de reservas con 10 niños es: 1


Como se puede observar hay valores atipicos que se alejan fuertemente de los datos, debido a la poca significancia cuantitativa de estos datos atipicos, se decide eliminar las observaciones que los contienen.

In [11]:
#SIGNIFICANCIA PORCENTUAL DE LOS DATOS A ELIMINAR
perc = (df.loc[df.num_children >3].shape[0]/df.shape[0])*100
print(f'Los datos a eliminar representan el {perc:.3f}% de los datos')

#SE ELIMINAN DATOS AOUTLIERS
df = df.loc[df.num_children <= 3]

Los datos a eliminar representan el 0.008% de los datos


In [12]:
df.num_children.describe()

count    36133.000000
mean         0.097113
std          0.375780
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          3.000000
Name: num_children, dtype: float64

### Variables ***weekend_nights*** y ***week_nights***

Estas variables describen la cantidad de dias de fin de semana y dias de semana respectivamente por la cual se reserva la habitacion del hotel. A partir de esto se generará una nueva variable que describa la cantidad de dias totales por la cual se reseva.

In [13]:
#SE CREA DATAFRAME ADICIONAL PARA CREAR NUEVA VARIABLE
booking_nights = df[['weekend_nights','week_nights']].copy()

#SE CREA NUEVA VARIABLE EN BASE A LAS EXISTENTES
booking_nights.loc[:,'total_nights'] = booking_nights.apply(lambda x: x.weekend_nights + x.week_nights, axis=1)

#INFORMACION DE LA NUEVA VARIABLE
booking_nights.total_nights.describe()

count    36133.000000
mean         3.013782
std          1.785856
min          0.000000
25%          2.000000
50%          3.000000
75%          4.000000
max         24.000000
Name: total_nights, dtype: float64

En la informacion que se muestra de la nueva variable se puede observar que el valor minimo en ella es **cero**, lo cual no seria valido. Por este motivo es que se realizara un analisis para aplicar alguna tecnica de imputacion para los valores atipicos mencionados.

In [14]:
#MEDIDAS DE TENDENCIA CENTRAL DE LA VARIABLE
print(f'Media: {booking_nights.total_nights.mean()}')
print(f'Mediana: {booking_nights.total_nights.median()}')
print(f'Moda: {stats.mode(booking_nights.total_nights)[0]}')

#MEDIDAS DE DISPERSION
print(f'Coef asimetria: {booking_nights.total_nights.skew()}')
print(f'Kurtosis: {booking_nights.total_nights.kurtosis()}')

Media: 3.013782414966928
Mediana: 3.0
Moda: 3
Coef asimetria: 2.235381600158218
Kurtosis: 12.25874810112899


In [15]:
#IMPUTACION DE DATOS ATIPICOS
booking_nights.loc[booking_nights.total_nights == 0, 'total_nights'] = booking_nights.total_nights.median()

#MEDIDAS DE DISPERSION
print(f'Coef asimetria: {booking_nights.total_nights.skew()}')
print(f'Kurtosis: {booking_nights.total_nights.kurtosis()}')

Coef asimetria: 2.2557593832576317
Kurtosis: 12.397887471427797


Como se observa en los coeficientes de dispersion, la imputacion por la **mediana** no ha impactado de manera violenta en la distribucion de la variable, es por ello que se conserva este cambio. Ahora solo queda por unir esta nueva variable al **Dataframe** principal.

In [16]:
#SE INSERTA LA NUEVA VARIABLE AL DATAFRAME PRINCIPAL
position = df.columns.get_loc('week_nights')+1
df.insert(column='total_nights', loc=position, value=booking_nights.total_nights)

#SE LIBERAN RECURSOS
del booking_nights

#INFORMACION
df.total_nights.describe()

count    36133.000000
mean         3.020258
std          1.780346
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max         24.000000
Name: total_nights, dtype: float64

### Variables ***arrival_year***, ***arrival_month*** y ***arrival_date***

Estas variables describen el año, mes y dia de arribo respectivamente. Lo que se hara en esta seccion es crear una variable especifica para describir la fecha y con esta informacion obtener la estacion del año para la cual se realizo la reserva

In [13]:
#PRiMERO SE REALIZARA UN CAMBIO DE NOMBRE A FINES DE EVITAR AMBIGUEDAD
df.rename(columns={'arrival_date':'arrival_day'}, inplace=True)

#SE CREA DATAFRAME COMPLEMETARIO PARA RELIZAR LOS CAMBIOS
dates = df[['arrival_year','arrival_month', 'arrival_day']].copy()
dates.columns = ['y','m','d']

#SE GENERA LA NUEVA VARIABLE
dates.loc[:,'arrival_date'] = dates.apply(lambda x: str(x.y) + '-' + str(x.m) + '-' + str(x.d), axis=1)
dates.head(3)

,y,m,d,arrival_date
0,2017,10,2,2017-10-2
1,2018,11,6,2018-11-6
2,2018,2,28,2018-2-28


Si bien ya se ha obtenido la columna de fecha, esta tiene un tipo de dato ***str***, lo cual no es nuestro objetivo. Por ello se transformaran estos datos a tipo ***Datetime*** lo que va a generar algunos datos nulos que manejaremos posteriormente

In [14]:
#SE CONVIERTE LA VARIABLE ARRIVAL_DATE EN DATETIME
dates.arrival_date = pd.to_datetime(dates.arrival_date, format="%Y-%m-%d",yearfirst=True, errors='coerce')
dates.arrival_date.info()

<class 'pandas.core.series.Series'>
Index: 36133 entries, 0 to 36274
Series name: arrival_date
Non-Null Count  Dtype         
--------------  -----         
36096 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 564.6 KB


In [15]:
#OBSERVACION DE DATOS NULOS
dates.loc[dates.arrival_date.isna()].head(3)

,y,m,d,arrival_date
2626,2018,2,29,NaT
3677,2018,2,29,NaT
5600,2018,2,29,NaT


Como se puede observar los datos ***NaN*** que se generaron al convertir de ***string*** a ***datetime*** fueron producidos por la fecha ***2018-02-29*** una fecha correspondiente a un año biciesto, pero justamente el año 2018 no lo fue. Para este caso en particular reemplazaremos las fechas conflictivas con la fecha del dia anterior.

In [16]:
#CORRECCION DEL ERROR
condition = (dates.y == 2018) & (dates.m == 2) & (dates.d == 29)
dates.loc[condition, 'd'] = 28

#CONVERSION DE TIPO
dates.loc[dates.arrival_date.isna(), 'arrival_date'] = dates.apply(lambda x: pd.to_datetime(str(x.y) + '-' + str(x.m) + '-' + str(x.d), format="%Y-%m-%d",yearfirst=True), axis=1)

#INFORMACION
dates.arrival_date.info()

<class 'pandas.core.series.Series'>
Index: 36133 entries, 0 to 36274
Series name: arrival_date
Non-Null Count  Dtype         
--------------  -----         
36133 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.6 MB


#### Variable ***arrival season***

Como se mencion anteriormente, se dara lugar a la creacion de esta variable, que refiere la estacion del año para la cual se realiza la reserva.

In [17]:
#CREANDO ESTACIONES DEL AÑO (EUROPA)

#SE CREAN LAS VARIABLES NECESARIAS
month = dates['arrival_date'].dt.month
date = dates['arrival_date'].dt.day

#se crean variables bcon condiciones a fin de establecer estaciones
is_summer = ((month == 6) & (date >= 21) | (month == 7) | (month == 8) | (month == 9) & (date <= 22))
is_autumn = ((month == 9) & (date >= 23) | (month == 10) | (month == 11) | (month == 12) & (date <= 20))
is_winter = ((month == 12) & (date >= 21) | (month == 1) | (month == 2) | (month == 3) & (date <= 20))

#Se crea la columna "Season" y se le asigna el valor "spring" para todos los valores de la columna
dates["arrival_season"] = "spring"

#Se asigna valores a la columna "season" segun la fecha
dates.loc[is_summer, "arrival_season"] = "summer"
dates.loc[is_autumn, "arrival_season"] = "autumn"
dates.loc[is_winter, "arrival_season"] = "winter"

dates.head(3)

,y,m,d,arrival_date,arrival_season
0,2017,10,2,2017-10-02,autumn
1,2018,11,6,2018-11-06,autumn
2,2018,2,28,2018-02-28,winter


In [18]:
#CREANDO ESTACIONES DEL AÑO (EUROPA)

#SE CREAN VARIABLES CONDICIONALES
is_summer = ((dates.m == 6) & (dates.d >= 21) | (dates.m == 7) | (dates.d == 8) | (dates.m == 9) & (dates.d <= 22))
is_autumn = ((dates.m == 9) & (dates.d >= 23) | (dates.m == 10) | (dates.d == 11) | (dates.m == 12) & (dates.d <= 20))
is_winter = ((dates.m== 12) & (dates.d >= 21) | (dates.m == 1) | (dates.d == 2) | (dates.m == 3) & (dates.d <= 20))

#SE CREA NUEVA VARIABLE
dates["arrival_season"] = "spring"
dates.loc[is_summer, "arrival_season"] = "summer"
dates.loc[is_autumn, "arrival_season"] = "autumn"
dates.loc[is_winter, "arrival_season"] = "winter"

dates.head(3)

,y,m,d,arrival_date,arrival_season
0,2017,10,2,2017-10-02,winter
1,2018,11,6,2018-11-06,spring
2,2018,2,28,2018-02-28,spring


Con el objetivo logrado parcialmente, ahora es momento de deshacerse de las columnas que no nos sirven en del dataframe principal y agregar las nuevas columnas que se consiguieron

In [19]:
#SE IGUALAN LAS COLUMNAS DEL DATAFRAME PRINCIPAL CON EL COMPLEMENTARIO DE LIBRE DE ERRORES
df.arrival_month = dates.m
df.arrival_day = dates.d

#SE INSERTA LA VARIABLE NUEVA EN EL DATRAFRAME PRINCIPAL
position = df.columns.get_loc('arrival_day')+1
df.insert(column='arrival_season', loc=position, value=dates.arrival_season)

#SE LIBERAN RECURSOS
del dates

df.head(3)

,Booking_ID,num_adults,num_children,weekend_nights,week_nights,meal_plan,car_parking,room_type,lead_time,arrival_year,arrival_month,arrival_day,arrival_season,mkt_segment,repeated_guest,num_prev_cancellations,num_prev_not_canceled,avg_price_room,num_sp_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,winter,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,spring,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,spring,Online,0,0,0,60.00,0,Canceled


### Variable ***avg_price_room***

In [ ]:
df.avg.

### Variables cualitativas

En esta seccion se le da tratamiento a quellas variables que no son numericas. Para ello se utilizara la herramienta ***LabelEncoder*** destinado a las variables nominales, mientras que para las ordinales se lo hara respetando el orden inherente a su naturaleza.

In [24]:
#Se crea una lista con el nombre de las variables de tipo object/str
qualitative_var = [column for column in df.columns if df[column].dtype == 'object']
print(qualitative_var)

['Booking_ID', 'meal_plan', 'room_type', 'arrival_season', 'mkt_segment', 'booking_status']


Para explorar el contenido de cada variable cualitativa se imprime us valores unicos para cada una de ellas

In [25]:
#Se remueve el ID ya que no es una variable cualitativa sino una un identificador
qualitative_var.remove('Booking_ID')

#Impresion de valores unicos
for var in qualitative_var:
    print(f'{var}:')
    for item in df[var].unique():
        print(f'\t{item}')

meal_plan:
	Meal Plan 1
	Not Selected
	Meal Plan 2
	Meal Plan 3
room_type:
	Room_Type 1
	Room_Type 4
	Room_Type 6
	Room_Type 5
	Room_Type 2
	Room_Type 7
	Room_Type 3
arrival_season:
	winter
	spring
	autumn
	summer
mkt_segment:
	Offline
	Online
	Corporate
	Aviation
	Complementary
booking_status:
	Not_Canceled
	Canceled


Conociendo mejor el contenido de las variables cualitativas, se decide crear las siguientes variables numericas:
- meal_plan_id
- room_type_id
- booking_status_id
- mkt_segment_id
- arrival_season_id

Cada una corresponde a una variable cualitativa pero conteniendo un codigo numerico correspondiente a cada valor unico de la variable. Para aquellas que son de naturaleza ordinal se asignan los valores cuantitativos de manera manual a fin de conservar el orden natural de la variable. Mientras que para las nominales se utilza ***LabelEncoder***.

In [26]:
df_qualitative = df[qualitative_var].copy()

#Variables ordinales
df_qualitative['meal_plan_id'] = df_qualitative['meal_plan']
df_qualitative['meal_plan_id'] = df_qualitative['meal_plan_id'].str.replace('Not Selected', '0')
df_qualitative['meal_plan_id'] = df_qualitative['meal_plan_id'].str.replace(r'.*1$', '1', regex=True)
df_qualitative['meal_plan_id'] = df_qualitative['meal_plan_id'].str.replace(r'.*2$', '2', regex=True)
df_qualitative['meal_plan_id'] = df_qualitative['meal_plan_id'].str.replace(r'.*3$', '3', regex=True)
df_qualitative['meal_plan_id'] = df_qualitative['meal_plan_id'].astype(int)

df_qualitative['room_type_id'] = df_qualitative['room_type']
df_qualitative['room_type_id'] = df_qualitative['room_type_id'].str.replace(r'.*1$', '1', regex=True)
df_qualitative['room_type_id'] = df_qualitative['room_type_id'].str.replace(r'.*2$', '2', regex=True)
df_qualitative['room_type_id'] = df_qualitative['room_type_id'].str.replace(r'.*3$', '3', regex=True)
df_qualitative['room_type_id'] = df_qualitative['room_type_id'].str.replace(r'.*4$', '4', regex=True)
df_qualitative['room_type_id'] = df_qualitative['room_type_id'].str.replace(r'.*5$', '5', regex=True)
df_qualitative['room_type_id'] = df_qualitative['room_type_id'].str.replace(r'.*6$', '6', regex=True)
df_qualitative['room_type_id'] = df_qualitative['room_type_id'].str.replace(r'.*7$', '7', regex=True)
df_qualitative['room_type_id'] = df_qualitative['room_type_id'].astype(int)

#Variables nominales
df_qualitative['booking_status_id'] = df_qualitative['booking_status']
df_qualitative['booking_status_id'] = df_qualitative['booking_status_id'].str.replace('Not_Canceled','0')
df_qualitative['booking_status_id'] = df_qualitative['booking_status_id'].str.replace('Canceled','1')
df_qualitative['booking_status_id'] = df_qualitative['booking_status_id'].astype(int)

encoder = LabelEncoder()
df_qualitative['mkt_segment_id'] = encoder.fit_transform(df_qualitative.mkt_segment.values)
df_qualitative['arrival_season_id'] = encoder.fit_transform(df_qualitative.arrival_season.values)

#Se eliminan variables que ya no necesitamos
df_qualitative.drop(columns=qualitative_var, inplace=True)

df_qualitative.head()

,meal_plan_id,room_type_id,booking_status_id,mkt_segment_id,arrival_season_id
0,1,1,0,3,3
1,0,1,0,4,1
2,1,1,1,4,1
3,1,1,1,4,1
4,0,1,1,4,0


Luego de crear las variables numericas se las insertan en el datafreme original

In [27]:
df.insert(column='meal_plan_id', loc=df.columns.get_loc('meal_plan')+1, value=df_qualitative.meal_plan_id)
df.insert(column='room_type_id', loc=df.columns.get_loc('room_type')+1, value=df_qualitative.room_type_id)
df.insert(column='booking_status_id', loc=df.columns.get_loc('booking_status')+1, value=df_qualitative.booking_status_id)
df.insert(column='mkt_segment_id', loc=df.columns.get_loc('mkt_segment')+1, value=df_qualitative.mkt_segment_id)
df.insert(column='arrival_season_id', loc=df.columns.get_loc('arrival_season')+1, value=df_qualitative.arrival_season_id)
del df_qualitative
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36133 entries, 0 to 36274
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Booking_ID              36133 non-null  object 
 1   num_adults              36133 non-null  int64  
 2   num_children            36133 non-null  int64  
 3   weekend_nights          36133 non-null  int64  
 4   week_nights             36133 non-null  int64  
 5   total_nights            36133 non-null  int64  
 6   meal_plan               36133 non-null  object 
 7   meal_plan_id            36133 non-null  int32  
 8   car_parking             36133 non-null  int64  
 9   room_type               36133 non-null  object 
 10  room_type_id            36133 non-null  int32  
 11  lead_time               36133 non-null  int64  
 12  arrival_year            36133 non-null  int64  
 13  arrival_month           36133 non-null  int64  
 14  arrival_day             36133 non-null  int

### Habitaciones con precio cero

Basado en el resultado del metodo ***describe()*** se puede ver que en la columna ***avg_price_room*** el minimo valor es cero y el precio de una habitacion no puede ser cero, asi que eliminaremos los registros con valor cero ya que a continuacion veremos que porcentaje de la informacion representan estas reservas.

In [28]:
df.avg_price_room.describe()

count    36133.000000
mean       103.509164
std         35.062636
min          0.000000
25%         80.390000
50%         99.480000
75%        120.120000
max        540.000000
Name: avg_price_room, dtype: float64

In [29]:
perc_price_cero = (df.loc[df.avg_price_room == 0].shape[0] / df.shape[0])*100
print(f'Cantidad de reservas con precio cero: {df.loc[df.avg_price_room == 0].shape[0]}')
print(f'Porcentaje de las reservas con precio cero en relacion al dataset principal: {perc_price_cero:.2f}%')

Cantidad de reservas con precio cero: 541
Porcentaje de las reservas con precio cero en relacion al dataset principal: 1.50%


In [30]:
df = df.loc[df.avg_price_room > 0]
df.avg_price_room.describe()

count    35592.000000
mean       105.082508
std         32.905017
min          0.500000
25%         81.000000
50%        100.000000
75%        121.000000
max        540.000000
Name: avg_price_room, dtype: float64

De esta manera descartamos los precios cero de las habitaciones  en las reservas, pero pudimos observar un valor minimo en la variable de 0.5 y por reglas de negocio el valor minimo aceptado en este campo es 10 EUR, por ende eliminaremos estas observaciones con valores que se interpretan como un error al momento de cargar la informacion.

In [31]:
perc_price_cero = (df.loc[df.avg_price_room < 10].shape[0] / df.shape[0])*100
print(f'Cantidad de reservas con precio cero: {df.loc[df.avg_price_room < 10].shape[0]}')
print(f'Porcentaje de las reservas con precio menor a 10 en relacion al dataset principal: {perc_price_cero:.2f}%')

Cantidad de reservas con precio cero: 40
Porcentaje de las reservas con precio menor a 10 en relacion al dataset principal: 0.11%


In [32]:
df = df.loc[df.avg_price_room >= 10]
df.avg_price_room.describe()

count    35552.000000
mean       105.196361
std         32.747768
min         12.000000
25%         81.000000
50%        100.000000
75%        121.000000
max        540.000000
Name: avg_price_room, dtype: float64

### Revision de outliers en variable ***avg_price_room***

Siguiendo con la variable ***avg_price_room*** en el ***describe*** anterior pudimos ver que el valor maximo de la variable se despega bastatante del percentil 75, por ende revisaremos si es que estos valores son atipicos o no.

In [33]:
df.avg_price_room.describe()

count    35552.000000
mean       105.196361
std         32.747768
min         12.000000
25%         81.000000
50%        100.000000
75%        121.000000
max        540.000000
Name: avg_price_room, dtype: float64

In [34]:
count = df.loc[df.avg_price_room > 300].shape[0]
perc = (count/df.shape[0])*100
print(f'La cantidad de precios por encima de 300 EUR es: {count}')
print(f'Estos {count} valores respresetan el {perc:.2f}% de los datos totales')

La cantidad de precios por encima de 300 EUR es: 9
Estos 9 valores respresetan el 0.03% de los datos totales


Viendo estos valores como outliers se deciden eliminarlos dada su minima cantidad y a su baja representacion porcentual de los datos

In [35]:
df = df.loc[df.avg_price_room < 300]
df.avg_price_room.describe()

count    35538.000000
mean       105.105415
std         32.408827
min         12.000000
25%         81.000000
50%        100.000000
75%        121.000000
max        299.330000
Name: avg_price_room, dtype: float64

### Variable objetivo ***booking_status***

En esta seccion se reemplazan los valores de la variable de interes solo para fines descriptivos, ya que al momento de entrenar el modelo tenemos la dicotomia intacta representada numericamente.

In [36]:
df.booking_status = df.booking_status.str.replace('Not_Canceled', 'No cancelada')
df.booking_status = df.booking_status.str.replace('Canceled', 'Cancelada')